In [ ]:
%load_ext tensorboard
import datetime

import tensorflow as tf
import numpy as np

import dataset
import gpu_utils
gpu_utils.setup_gpus()

BUFFER_SIZE = 10000
BATCH_SIZE = 64  # 512

In [ ]:
data = dataset.load_file(BATCH_SIZE, 0.2, filenames=['bible_text/bible.txt','short_table/short_table.txt'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(256, return_sequences=True, recurrent_initializer='glorot_uniform'),
        input_shape=(data.maxlen, data.letters_size),
        batch_size=BATCH_SIZE
    ),
    tf.keras.layers.GRU(data.niqqud_size, return_sequences=True, recurrent_initializer='glorot_uniform'),
])

# best: mean_squared_logarithmic_error, stateless

model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])
model.build()

model.summary()

In [ ]:
model.load_weights(tf.train.latest_checkpoint('niqqud_checkpoints/'))

In [ ]:
%tensorboard --logdir logs\fit

In [ ]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(data.input_texts, data.niqqud_texts,
          batch_size=BATCH_SIZE,
          epochs=2,
          validation_data=(data.input_validation, data.niqqud_validation),
          callbacks=[
              # tf.keras.callbacks.ModelCheckpoint(filepath='niqqud_checkpoints/ckpt_{epoch}', save_weights_only=True),
              tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
              tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=0, min_lr=0.001),
              tensorboard_callback,
          ]
          )

In [ ]:
model.add(tf.keras.layers.Softmax())

def print_predictions(batch):
    results = data.merge(batch, model.predict(batch))

    for r in results:
        print(r)


print_predictions(data.input_texts[0:BATCH_SIZE])

In [ ]:
del model